In [29]:
from typing import List
import pandas as pd
import json
import os

def compute_validation_effectiveness(baseline_file: str, model_names: List, result_dir: str, index_name: str):

    df_baseline = pd.read_csv(baseline_file)
    
    results = []

    for model_name in model_names:
        
        results_file = f"{result_dir}/all_dependencies_{index_name}_{model_name}.json"

        with open(results_file, "r", encoding="utf-8") as src:
            data = json.load(src)

        true_positives = []
        true_negatives = []
        false_positives = []
        false_negatives = []
        accuracy_count = []
        skipped = 0

        for entry, (index, row) in zip(data, df_baseline.iterrows()):

            response_dict = None
            isDependency = None

            rating = row["final_rating"]
            try:
                if "responses" in entry:
                    response = entry["responses"][0]
                else:
                    response = entry["response"]
                response_dict = json.loads(response, strict=False)
                isDependency = response_dict["isDependency"]
            except (json.JSONDecodeError, KeyError):
                skipped += 1
                continue
                
            if str(rating) == "Borderline":
                if isDependency:
                    accuracy_count.append(1)
                    true_positives.append(1)
                if not isDependency:
                    accuracy_count.append(1)
                    true_negatives.append(1)

            # TP: The LLM validates a dependency as correct and the dependency is correct
            if isDependency and str(rating).lower() == "true":
                accuracy_count.append(1)
                true_positives.append(1)
                
            # FP: The LLM validates a dependency as correct, but the dependency is actually incorrect
            if isDependency and str(rating).lower() == "false":
                accuracy_count.append(0)
                false_positives.append(1)

            # TN: The LLM validates a dependency as incorrect and the dependency is incorrect
            if not isDependency and str(rating).lower() == "false":
                accuracy_count.append(1)
                true_negatives.append(1)

            # FN: The LLM validates a dependency as incorrect, but the dependency is actually correct
            if not isDependency and  str(rating).lower() == "true":
                accuracy_count.append(0)
                false_negatives.append(1)

        
        tp = sum(true_positives)
        fp = sum(false_positives)
        fn = sum(false_negatives)
        tn = sum(true_negatives)
        accuracy = sum(accuracy_count)/len(accuracy_count)


        print("Model: ", model_name)
        print("File: ", results_file)
        print("TP", tp)
        print("FP", fp)
        print("TN", tn)
        print("FN", fn)
        print("Sum correct response format: ", sum([tp, fp, tn, fn]))
        print("Entries skipped: ", skipped)

        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1_score = 2 * (precision * recall) / (precision + recall)

        print("Accuracy", accuracy)
        print("Precision", precision)
        print("Recall", recall)
        print("F1 Score: ", f1_score)

        results.append({
            "model_name": model_name,
            "file": results_file,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1-score": f1_score
        })
    

    df_results = pd.DataFrame(results)
    df_results.to_csv(f"{result_dir}/validation_effectiveness.csv", index=False)


compute_validation_effectiveness(
    baseline_file="../data/results/all_dependencies.csv",
    model_names=["gpt-4o-2024-05-13", "gpt-3.5-turbo-0125", "llama3:70b", "llama3:8b"],
    result_dir="../data/results/without",
    index_name="without"
)

Model:  gpt-4o-2024-05-13
File:  ../data/results/without/all_dependencies_without_gpt-4o-2024-05-13.json
TP 127
FP 15
TN 282
FN 76
Sum correct response format:  500
Entries skipped:  0
Accuracy 0.818
Precision 0.8943661971830986
Recall 0.625615763546798
F1 Score:  0.7362318840579711
Model:  gpt-3.5-turbo-0125
File:  ../data/results/without/all_dependencies_without_gpt-3.5-turbo-0125.json
TP 137
FP 88
TN 209
FN 66
Sum correct response format:  500
Entries skipped:  0
Accuracy 0.692
Precision 0.6088888888888889
Recall 0.6748768472906403
F1 Score:  0.6401869158878505
Model:  llama3:70b
File:  ../data/results/without/all_dependencies_without_llama3:70b.json
TP 112
FP 63
TN 234
FN 91
Sum correct response format:  500
Entries skipped:  0
Accuracy 0.692
Precision 0.64
Recall 0.5517241379310345
F1 Score:  0.5925925925925927
Model:  llama3:8b
File:  ../data/results/without/all_dependencies_without_llama3:8b.json
TP 158
FP 126
TN 158
FN 32
Sum correct response format:  474
Entries skipped:  26
A

In [30]:
import pandas as pd

file_path = "../data/results/without/validation_effectiveness.csv"

df = pd.read_csv(file_path)

df

,model_name,file,accuracy,precision,recall,f1-score
0,gpt-4o-2024-05-13,../data/results/without/all_dependencies_witho...,0.818000,0.894366,0.625616,0.736232
1,gpt-3.5-turbo-0125,../data/results/without/all_dependencies_witho...,0.692000,0.608889,0.674877,0.640187
2,llama3:70b,../data/results/without/all_dependencies_witho...,0.692000,0.640000,0.551724,0.592593
3,llama3:8b,../data/results/without/all_dependencies_witho...,0.666667,0.556338,0.831579,0.666667
